In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 23.0 MB/s eta 0:00:00


### 라이브러리

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time
from datetime import datetime
import logging
import json
import os

### Selenium WebDriver 설정

In [17]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--remote-debugging-port=9222')

driver = webdriver.Chrome(options=chrome_options)

### 로깅 설정

In [18]:
log_dir = '/content/drive/MyDrive/[인공지능기초]데이터크롤링/news/fox'
log_file = os.path.join(log_dir, 'crawler.log')

logging.basicConfig(filename=log_file, level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

### 카테고리 및 날짜 범위 설정

In [19]:
CATEGORY = ["executive", "senate", "house-of-representatives", "judiciary", "foreign-policy"] #[행정부, 상원, 하원, 사법부, 외교 정책]

START_DATE = datetime(2024, 7, 21)
END_DATE = datetime(2024, 11, 5)

### 뉴스 데이터 파일 경로 설정

In [20]:
data_file = os.path.join(log_dir, 'news_data.json')

### 이미 수집된 데이터 불러오기

In [21]:
if os.path.exists(data_file):
    with open(data_file, 'r') as f:
        news_url = json.load(f)
else:
    news_url = []

total_articles_collected = len(news_url)
logging.info(f"Starting crawl. Already collected {total_articles_collected} articles.")

### 카테고리별 뉴스 URL, TITLE 수집

In [22]:
try:
    for cname in CATEGORY:
        try:
            print(f"- '{cname}' 카테고리 수집 시작")
            logging.info(f"Starting category '{cname}'")
            url = f"https://www.foxnews.com/category/politics/{cname}"
            driver.get(url)
            time.sleep(2)  # 페이지 로드 대기

            has_more = True
            page_count = 0  # "Show More" 클릭 횟수 추적
            while has_more:
                # 페이지 끝까지 스크롤하여 기사 로드
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)  # 기사 로드 대기

                # 마지막 기사 날짜 확인
                articles = driver.find_elements(By.CSS_SELECTOR, "div.content.article-list > article.article")
                if articles:
                    last_article = articles[-1]
                    date_element = last_article.find_element(By.CSS_SELECTOR, "div.meta span.time")
                    date_text = date_element.text

                    # 날짜에 연도 추가하여 파싱
                    try:
                        # 연도를 추가하여 날짜 파싱 (현재 연도를 사용)
                        last_pub_date = datetime.strptime(date_text + " " + str(START_DATE.year), "%B %d %Y")
                    except ValueError:
                        last_pub_date = datetime.now()

                    if last_pub_date < START_DATE:
                        print("    시작 날짜 이전의 기사가 나왔습니다. 더 이상 수집하지 않습니다.")
                        logging.info(f"Reached articles before START_DATE in category '{cname}'. Stopping.")
                        break

                try:
                    # "Show More" 버튼 클릭
                    show_more = driver.find_element(By.CSS_SELECTOR, ".js-load-more a")
                    driver.execute_script("arguments[0].click();", show_more)
                    time.sleep(2)  # 새로운 기사 로드 대기
                    page_count += 1
                    print(f"    '{cname}' 카테고리에서 'Show More' 버튼을 {page_count}번 클릭했습니다.")
                    logging.info(f"'Show More' clicked {page_count} times in category '{cname}'")
                except NoSuchElementException:
                    has_more = False  # 더 이상 로드할 기사 없음
                    logging.info(f"No more articles to load in category '{cname}'.")

            # 날짜 범위 내의 기사 수집
            articles = driver.find_elements(By.CSS_SELECTOR, "div.content.article-list > article.article:not(.vendor-unit)")
            for article in articles:
                try:
                    title_element = article.find_element(By.CSS_SELECTOR, "h4.title a")
                    title = title_element.text
                    url = title_element.get_attribute("href")
                    date_element = article.find_element(By.CSS_SELECTOR, "div.meta span.time")
                    date_text = date_element.text

                    # 날짜 변환: 연도를 추가하여 날짜 파싱
                    try:
                        pub_date = datetime.strptime(date_text + " " + str(START_DATE.year), "%B %d %Y")
                    except ValueError:
                        pub_date = datetime.now()

                    if pub_date < START_DATE or pub_date > END_DATE:
                        continue

                    # 중복 확인
                    if any(d['url'] == url for d in news_url):
                        continue  # 이미 수집된 기사

                    news_url.append({"title": title, "url": url, "category": cname, "date": pub_date.strftime("%Y-%m-%d")})
                    print(f"        url: {url}\n        title: {title}\n        date: {pub_date}\n")
                except NoSuchElementException:
                    continue  # 요소를 찾지 못하면 스킵

            print(f"    '{cname}' 카테고리에서 수집된 기사 수: {len(news_url) - total_articles_collected}\n")
            logging.info(f"Collected {len(news_url) - total_articles_collected} articles in category '{cname}'")
            total_articles_collected = len(news_url)

            # 수집한 데이터 저장
            with open(data_file, 'w') as f:
                json.dump(news_url, f, ensure_ascii=False, indent=4)

        except Exception as e:
            logging.exception(f"Error occurred while processing category '{cname}': {e}")
            # 수집된 데이터 저장
            with open(data_file, 'w') as f:
                json.dump(news_url, f, ensure_ascii=False, indent=4)
            continue  # 다음 카테고리로 진행

except Exception as e:
    logging.exception(f"An unexpected error occurred: {e}")
finally:
    driver.quit()
    print(f"총 수집된 기사 수: {len(news_url)}")
    logging.info(f"Crawling finished. Total articles collected: {len(news_url)}")
    # 수집된 데이터 저장
    with open(data_file, 'w') as f:
        json.dump(news_url, f, ensure_ascii=False, indent=4)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        url: https://www.foxnews.com/politics/trump-rips-house-republican-who-voted-impeach-him-message-endorsing-his-rival
        title: Trump rips House Republican who voted to impeach him in message endorsing his rival
        date: 2024-10-29 00:00:00

        url: https://www.foxnews.com/politics/pennsylvania-house-race-5-former-house-republicans-say-dont-support-rep-scott-perry
        title: Pennsylvania House race: 5 former House Republicans say don't support Rep Scott Perry
        date: 2024-10-29 00:00:00

        url: https://www.foxnews.com/politics/media-dems-compare-historic-trump-msg-rally-nazi-event-ignore-democrat-events-held
        title: Media, Dems compare historic Trump MSG rally to 'Nazi' event, ignore Democrat events held there
        date: 2024-10-28 00:00:00

        url: https://www.foxnews.com/video/6363873059112
        title: Handful of races in blue state could decide control of House
        date: 2024-10-28 00:00:0

### 수집한 기사 URL, TITLE 요약

In [25]:
from collections import defaultdict

# 수집된 데이터 파일 로드
data_file = '/content/drive/MyDrive/[인공지능기초]데이터크롤링/news/fox/news_data.json'

# 데이터 불러오기
try:
    with open(data_file, 'r') as f:
        news_url = json.load(f)
except FileNotFoundError:
    print("Error: Data file not found.")
    news_url = []

# 카테고리별 기사 개수 요약
category_summary = defaultdict(int)
total_articles = len(news_url)

# 날짜 범위 설정
START_DATE = datetime(2024, 7, 21)
END_DATE = datetime(2024, 11, 5)

# 날짜 범위 내 기사 개수
articles_in_range = 0

for article in news_url:
    # 카테고리별 기사 개수 증가
    category_summary[article['category']] += 1

    # 날짜 범위 내 기사 확인
    pub_date = datetime.strptime(article['date'], "%Y-%m-%d")
    if START_DATE <= pub_date <= END_DATE:
        articles_in_range += 1

# 요약 출력
print(f"총 수집된 기사 수: {total_articles}")
print(f"날짜 범위 내 기사 수 ({START_DATE.strftime('%Y-%m-%d')} - {END_DATE.strftime('%Y-%m-%d')}): {articles_in_range}")
print("\n카테고리별 기사 수:")
for category, count in category_summary.items():
    print(f"  {category}: {count}개")

# 한 달별 기사 수 카운트
month_summary = defaultdict(int)

for article in news_url:
    pub_date = datetime.strptime(article['date'], "%Y-%m-%d")

    # 날짜 범위 내에서만 카운트
    if START_DATE <= pub_date <= END_DATE:
        # "YYYY-MM" 형식으로 월을 추출하여 카운트
        month_key = pub_date.strftime("%Y-%m")
        month_summary[month_key] += 1

# 한 달별 기사 수 출력
print("\n한 달별 기사 수:")
for month, count in sorted(month_summary.items()):
    print(f"  {month}: {count}개")


총 수집된 기사 수: 2201
날짜 범위 내 기사 수 (2024-07-21 - 2024-11-05): 2201

카테고리별 기사 수:
  executive: 661개
  senate: 283개
  house-of-representatives: 390개
  judiciary: 304개
  foreign-policy: 563개

한 달별 기사 수:
  2024-07: 311개
  2024-08: 633개
  2024-09: 556개
  2024-10: 602개
  2024-11: 99개
